In [3]:
## Helper Functions

suits = ['s', 'h', 'd', 'c']
vals = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']

import time 
import numpy as np
import itertools as it
import deuces as d #God bless jwg4

def win_prob(your_hand, river_cards, no_of_other_hands, sim=10000):
    """
    Input Example:
        Your Hand: ['Ah', '7c'] ---> Ace of Hearts and Seven of Clubs
        River Cards: ['Ad', '2d', '6d'] --> Ace of Diamonds, 2 of Diamonds, 6 of Diamonds, and 2 others (always 5)
        No of Others: 2 ---> 2 other players remain
        Simulations: number of Monte Carlo simulations to run, default=10000
    Output: Your win %
        If river_cards are present: Use MC simulations to estimate odds
        If no river_cards are present: Use lookup table to find pre-flop odds and return immediately
    """
    possible_cards = [a+b for a in vals for b in suits]
    for card in your_hand:
        possible_cards.remove(card)
    for card in river_cards:
        possible_cards.remove(card)
        
    wins = 0
    num_cards = 5 - len(river_cards) + 2 * no_of_other_hands
    evaluator = d.Evaluator()
    
    for i in range(sim): #10,000 MC simulations
        generated_cards = np.random.choice(possible_cards, num_cards, replace=False)
        counter = 0
        my_hand = [d.Card.new(card) for card in your_hand]
        board = [d.Card.new(card) for card in river_cards]
        while len(board) < 5:
            board.append(d.Card.new(generated_cards[counter]))
            counter += 1
        hand_strength = evaluator.evaluate(board, my_hand)
        best_strength = 1e10 #some random large number
        no_best_hands = 1 #number of hands that are at best_strength
        for j in range(no_of_other_hands):
            new_hand = [d.Card.new(generated_cards[counter]), d.Card.new(generated_cards[counter+1])]
            counter += 2
            new_strength = evaluator.evaluate(board, new_hand)
            if new_strength < best_strength:
                best_strength = new_strength
                no_best_hands = 1 #reset
            elif new_strength == best_strength:
                no_best_hands += 1 #add one more
        if best_strength > hand_strength:
            wins += 1
        elif best_strength == hand_strength:
            wins += 1/no_best_hands

    return wins/sim
    
a = time.time()
wp = win_prob(['Ah', 'Ac'], ['Ad', '2c', '6h'], 5, sim=10000)
b = time.time()
print("Time taken: ", b-a, "Win %: ", wp)

## This should really be adjusted by priors and also betting history to eliminate/down-weight unlikely hands.

Time taken:  4.004253387451172 Win %:  0.8928


In [4]:
def print_hand(hand):
    """
    Input: A hand, for example ['Ah', 'Ac']
    Output: A pretty string version of the hand
    """
    print

class PokerTable(object):
    def __init__(self, players, startingStack, smallBlind, dealer):
        """
        Input: 
            No of Players: [P1, P2, P3] -> 3 players, each of which is a PokerAI object.
            startingStack: 100 -> 100 chips to start
            smallBlind: 1 -> small blind of 1, big blind assumed to be 2
            dealer: 0 -> first player (in player array) assumed to be the dealer. Left is small, then blind.
        """
        self.dealer = 0
        self.smallBlind = smallBlind
        self.bigBlind = smallBlind * 2
        self.stacks = [startingStack] * len(players)
        self.players = players 
        
        self.suits = ['s', 'h', 'd', 'c']
        self.vals = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
        
    def playRound(self):
        """
        Plays one round and updates parameters
        Betting Format:
            raise int1 -> raise by int1. Example: raise 2
            check -> raise by 0
            call -> raise by the expected amount
            fold -> exit position
        """
        # Initialise board and hands. Deal out the first two hands and open the betting
        board = [] # Format: list of Cards
        hands = [] # Format: list of (player #, [Card1, Card2])
        possible_cards = [a+b for a in self.vals for self.b in suits]
        
        for player in range(len(self.players)):
            cards = np.random.choice(possible_cards, 2, replace=False)
            for card in cards:
                possible_cards.remove(card)
            hands.append((player, [card[0], card[1]]))
            
        print("The round has begun and the cards have been dealt.")
        
        # Play each of the 4 betting rounds
        print("We now move onto the pre-flop phase of betting.")
        bets = [0] * len(self.players)
        bets[self.dealer+1] = self.smallBlind
        bets[self.dealer+2] = self.bigBlind
        players_still_in = range(len(self.players)) 
        min_bet = self.bigBlind
        
        # Pre-flop betting
        for i in range(len(self.players)):
            turn = (self.dealer + i + 3) % len(self.players) #whose turn is it?
            print("It is now player {0}'s turn".format(turn))
            move = self.players[turn].move()
            print("Player {0} plays: {1}".format(turn, move))
            if move == "fold":
                players_still_in.remove(turn)
            if move == "call":
                self.stacks[turn] -= 
            
        for rnd in ['Flop', 'Turn', 'River']:
            
        
        # Judge the winner of the round
        best_hand_strength = 1e10
        for hand in hands:
            
        
        # End the round by printing all hands, adjusting stacks and moving the dealer over
        self.dealer = (self.dealer + 1) % len(self.players)
        
        
        
class PokerAI(object):
    """
    Generic PokerAI class
    """
    
class HeuristicAI(PokerAI):
    """
    A HeuristicAI that plays based on heuristics (pot odds, betting history + hyperparams) and CFR.
    Hyperparameters are: betting aggression and bluffing %
    Store priors on enemies??? (probably greatly improve performance)
    """
    def __init__(self, def_prob, agg=1):
        """
        Input: Various hyperparameters that govern play
            Aggression: How aggressively the bot bids
            Default Prob: Default probabil
            ity of folding, calling, raising or bluffing at
                          the buckets:
                          - RR <= 0.7 (struggling hand)
                          - 0.7 < R <= 0.9 (bad hand)
                          - 0.9 < R <= 1.1 (average hand)
                          - 1.1 < R <= 1.3 (good hand)
                          - 1.3 <= R (excellent hand)
            Example:  [
                [0.6, 0.2, 0.0, 0.2],
                [0.4, 0.4, 0.1, 0.1],
                [0.1, 0.7, 0.2, 0.0],
                [0.0, 0.6, 0.4, 0.0],
                [0.0, 0.3, 0.7, 0.0]
            ]
        """
        self.aggression = agg
        self.default_prob = def_prob
        self.vals = ['s', 'h', 'd', 'c']
        self.suits = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
        
    def move(your_hand, river_cards, players, pot, min_bet, sim=10000, history=False):
        """
        Input Example:
            Your Hand: ['Ah', '7c'] ---> Ace of Hearts and Seven of Clubs
            River Cards: ['Ad', '2d', '6d'] --> Ace of Diamonds, 2 of Diamonds, 6 of Diamonds, and 2 others (always 5)
            Players: [P1, P2, P3] -> the 3 players who are still in the round.
            Pot: 665 -> current pot of 665 
            Min Bet: 25 -> minimum bet to call, of 25.
            Simulations: number of Monte Carlo simulations to run, default=10000
            History: the historical betting data of the game, default=False (Implement this later.)
        Output: Your move as a string, in the following format:
            ('raise', 50) -> raise by 50 on top of 25 for a total bet of 75
            'call' -> raise by the expected amount, 25 (same as check)
            'fold' -> exit position
        """
        wp = self.win_prob(your_hand, river_cards, len(players)-1, sim)
        pot_odds = min_bet/(pot+min_bet)
        
        rr = wp/pot_odds # our main heuristic, expected rate of return
        prob = self.default_prob[min_index(abs([0.6, 0.8, 1.0, 1.2, 1.4] - rr))]
        ## We should adjust this probability with our rr!!!

        move = np.random.choice(['fold', 'call', 'raise', 'bluff'], prob)
        if move == "raise":
            raise_amount = pot / 3 * self.agg
            ## We should adjust this raise with our rr!!!
            return ("raise", int(raise_amount))
        elif move == "bluff":
            raise_amount = pot / 2 * self.agg
            ## We should adjust this raise with our rr!!!
            return ("raise", int(raise_amount))
        else:
            return move
    
    def win_prob(your_hand, river_cards, no_of_other_hands, sim=10000):
        """
        Input Example:
            Your Hand: ['Ah', '7c'] ---> Ace of Hearts and Seven of Clubs
            River Cards: ['Ad', '2d', '6d'] --> Ace of Diamonds, 2 of Diamonds, 6 of Diamonds, and 2 others (always 5)
            No of Others: 2 ---> 2 other players remain
            Simulations: number of Monte Carlo simulations to run, default=10000
        Output: Your win %
            If river_cards are present: Use MC simulations to estimate odds
            If no river_cards are present: Use lookup table to find pre-flop odds and return immediately
        """
        if len(river_cards) == 0: #Pre-flop, compute with lookup table
            return win_prob_lookup[your_hand]
            
        else:
            possible_cards = [a+b for a in vals for b in suits]
            for card in your_hand:
                possible_cards.remove(card)
            for card in river_cards:
                possible_cards.remove(card)

            wins = 0
            num_cards = 5 - len(river_cards) + 2 * no_of_other_hands
            evaluator = d.Evaluator()

            for i in range(sim): #10,000 MC simulations
                generated_cards = np.random.choice(possible_cards, num_cards, replace=False)
                counter = 0
                my_hand = [d.Card.new(card) for card in your_hand]
                board = [d.Card.new(card) for card in river_cards]
                while len(board) < 5:
                    board.append(d.Card.new(generated_cards[counter]))
                    counter += 1
                hand_strength = evaluator.evaluate(board, my_hand)
                best_strength = 1e10 #some random large number
                no_best_hands = 1 #number of hands that are at best_strength
                for j in range(no_of_other_hands):
                    new_hand = [d.Card.new(generated_cards[counter]), d.Card.new(generated_cards[counter+1])]
                    counter += 2
                    new_strength = evaluator.evaluate(board, new_hand)
                    if new_strength < best_strength:
                        best_strength = new_strength
                        no_best_hands = 1 #reset
                    elif new_strength == best_strength:
                        no_best_hands += 1 #add one more
                if best_strength > hand_strength:
                    wins += 1
                elif best_strength == hand_strength:
                    wins += 1/no_best_hands

            return wins/sim
    
class Human(PokerAI):
    """
    A human who inputs from cmd line
    """
    def __init__(self):
        print("Human Player initialised.")
    
    def move(your_hand, river_cards, players, pot, min_bet, sim=10000, history=False):
        print("Your hand is: ")
        print_hand(your_hand)
        move = raw_input("What is your move? Options are raise x, call or fold, for positive integer x.")
        if move == "call" or move == "fold":
            return move
        elif move[:5] == "raise":
            return ("raise", move.split(" ")[1])
        
v1 = HeuristicAI()

SyntaxError: invalid syntax (<ipython-input-4-f9d7dd2172a6>, line 65)

In [8]:
from deuces import Card
card = Card.new('Qh')

In [7]:
## Helper functions for conversion between the two libraries: Deuces and PyPoker.
import deuces as d

def pp_to_array(hand):
    return [card[1] + card[0].lower() for card in hand]

def pp_to_deuces(hand):
    return [d.Card.new(card[1] + card[0].lower()) for card in hand]

## Pre-process Preflop Odds
import pandas as pd
preflop_eq = pd.read_csv('preflop_equity.csv')
preflop_eq['Cards'] = [str(x) for x in preflop_eq['Cards']]
preflop_eq.set_index('Cards')

def preflop(cards, n):
    """
    Input:
        cards -- in size order, with either suited or unsuited: "AKo" and "KAo"
        num of players -- 3
    Output: Preflop win probability
    """
    return float(preflop_eq.loc[(preflop_eq['Cards'] == cards[0]) | (preflop_eq['Cards'] == cards[1])][str(n) + ' plyrs'].tolist()[0].split('%')[0])/100

#TEST: pp_to_array(['DJ', 'H6', 'S6', 'H5'])
preflop(['AKo', 'KAo'], 5)

0.324

In [31]:
from pypokerengine.players import BasePokerPlayer

class HeuristicPlayer(BasePokerPlayer):
    def __init__(self, def_prob, agg=1):
        """
        Input: Various hyperparameters that govern play
            Aggression: How aggressively the bot bids
            Default Prob: Default probabil
            ity of folding, calling, raising or bluffing at
                          the buckets:
                          - RR <= 0.7 (struggling hand)
                          - 0.7 < R <= 0.9 (bad hand)
                          - 0.9 < R <= 1.1 (average hand)
                          - 1.1 < R <= 1.3 (good hand)
                          - 1.3 <= R (excellent hand)
            Example:  [
                [0.6, 0.2, 0.0, 0.2],
                [0.4, 0.4, 0.1, 0.1],
                [0.1, 0.7, 0.2, 0.0],
                [0.0, 0.6, 0.4, 0.0],
                [0.0, 0.3, 0.7, 0.0]
            ]
        """
        self.aggression = agg
        self.default_prob = def_prob
        self.vals = ['s', 'h', 'd', 'c']
        self.suits = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
        
    def win_prob(self, your_hand, river_cards, no_of_other_hands, sim=10000):
        """
        Input Example:
            Your Hand: ['Ah', '7c'] ---> Ace of Hearts and Seven of Clubs
            River Cards: ['Ad', '2d', '6d'] --> Ace of Diamonds, 2 of Diamonds, 6 of Diamonds, and 2 others (always 5)
            No of Others: 2 ---> 2 other players remain
            Simulations: number of Monte Carlo simulations to run, default=10000
        Output: Your win %
            If river_cards are present: Use MC simulations to estimate odds
            If no river_cards are present: Use lookup table to find pre-flop odds and return immediately
        """
        if len(river_cards) == 0: #Pre-flop, compute with lookup table
            if your_hand[0][1] == your_hand[1][1]: #suited
                return preflop([your_hand[0][0] + your_hand[1][0] + 's', your_hand[1][0] + your_hand[0][0] + 's'], no_of_other_hands+1)
            else: #unsuited
                return preflop([your_hand[0][0] + your_hand[1][0] + 'o', your_hand[1][0] + your_hand[0][0] + 'o'], no_of_other_hands+1)
        else:
            possible_cards = [a+b for a in vals for b in suits]
            for card in your_hand:
                possible_cards.remove(card)
            for card in river_cards:
                possible_cards.remove(card)

            wins = 0
            num_cards = 5 - len(river_cards) + 2 * no_of_other_hands
            evaluator = d.Evaluator()

            for i in range(sim): #10,000 MC simulations
                generated_cards = np.random.choice(possible_cards, num_cards, replace=False)
                counter = 0
                my_hand = [d.Card.new(card) for card in your_hand]
                board = [d.Card.new(card) for card in river_cards]
                while len(board) < 5:
                    board.append(d.Card.new(generated_cards[counter]))
                    counter += 1
                hand_strength = evaluator.evaluate(board, my_hand)
                best_strength = 1e10 #some random large number
                no_best_hands = 1 #number of hands that are at best_strength
                for j in range(no_of_other_hands):
                    new_hand = [d.Card.new(generated_cards[counter]), d.Card.new(generated_cards[counter+1])]
                    counter += 2
                    new_strength = evaluator.evaluate(board, new_hand)
                    if new_strength < best_strength:
                        best_strength = new_strength
                        no_best_hands = 1 #reset
                    elif new_strength == best_strength:
                        no_best_hands += 1 #add one more
                if best_strength > hand_strength:
                    wins += 1
                elif best_strength == hand_strength:
                    wins += 1/no_best_hands

            return wins/sim
        
    def declare_action(self, valid_actions, hole_card, round_state):
        your_hand = pp_to_array(hole_card)
        river_cards = pp_to_array(round_state['community_card'])
        player_no = round_state['next_player'] #your position
        
        players_still_in = 0
        for player in round_state['seats']:
            if player['state'] == "participating":
                players_still_in += 1
        
        pot = round_state['pot']['main']['amount']
        if len(round_state['pot']) == 2:
            for sidepot in round_state['pot']['side']:
                pot += sidepot['amount'] #can always assume you are in sidepot, else you have no choices anyways.
            
        min_bet = valid_actions[1]['amount']
        stack = round_state['seats'][player_no]
        min_raise = valid_actions[2]['amount']['min']
        max_raise = valid_actions[2]['amount']['max']
        
        if min_bet == 0: #when we are first to act
            wp = self.win_prob(your_hand, river_cards, players_still_in-1)
            rr = wp * (players_still_in+1)
            prob = self.default_prob[np.argmin(abs(np.array([0.6, 0.8, 1.0, 1.2, 1.4]) - rr))]
            prob[1] = prob[0] + prob[1]
            prob[0] = 0 #we don't ever want to fold when we don't have to!!
            
        else:
            # Use heuristic to decide optimal move.
            wp = self.win_prob(your_hand, river_cards, players_still_in-1)
            pot_odds = min_bet/(pot+min_bet)

            rr = wp/pot_odds # our main heuristic, expected rate of return
            prob = self.default_prob[np.argmin(abs(np.array([0.6, 0.8, 1.0, 1.2, 1.4]) - rr))]
            print(prob)
            ## We should adjust this probability with our rr!!!

        move = np.random.choice(['fold', 'call', 'raise', 'bluff'], p=prob)
        if move == "raise":
            raise_amount = pot / 3 * self.aggression
            ## We should adjust this raise with our rr!!!
            raise_amount = int(max(min(raise_amount, max_raise), min_raise))
            if len(valid_actions) == 3: #is possible to raise, i.e. have enough money to raise
                return ("raise", raise_amount)
            else:
                return ("call", min_bet)
        elif move == "bluff":
            raise_amount = pot / 2 * self.aggression
            ## We should adjust this raise with our rr!!!
            raise_amount = int(max(min(raise_amount, max_raise), min_raise))
            if len(valid_actions) == 3: #is possible to raise, i.e. have enough money to raise
                return ("raise", raise_amount)
            else:
                return ("call", min_bet)
        elif move == "call":
            return (move, min_bet)
        else:
            return (move, 0)

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass
    
## Console Player

import pypokerengine.utils.visualize_utils as U

class ConsolePlayer(BasePokerPlayer):
    def __init__(self, input_receiver=None):
        self.input_receiver = input_receiver if input_receiver else self.__gen_raw_input_wrapper()

    def declare_action(self, valid_actions, hole_card, round_state):
        print(U.visualize_declare_action(valid_actions, hole_card, round_state, self.uuid))
        action, amount = self.__receive_action_from_console(valid_actions)
        return action, amount

    def receive_game_start_message(self, game_info):
        print(U.visualize_game_start(game_info, self.uuid))
        self.__wait_until_input()

    def receive_round_start_message(self, round_count, hole_card, seats):
        print(U.visualize_round_start(round_count, hole_card, seats, self.uuid))
        self.__wait_until_input()

    def receive_street_start_message(self, street, round_state):
        print(U.visualize_street_start(street, round_state, self.uuid))
        self.__wait_until_input()

    def receive_game_update_message(self, new_action, round_state):
        print(U.visualize_game_update(new_action, round_state, self.uuid))
        self.__wait_until_input()

    def receive_round_result_message(self, winners, hand_info, round_state):
        print(U.visualize_round_result(winners, hand_info, round_state, self.uuid))
        self.__wait_until_input()

    def __wait_until_input(self):
        input("Enter some key to continue ...")

    def __gen_raw_input_wrapper(self):
        return lambda msg: input(msg)

    def __receive_action_from_console(self, valid_actions):
        flg = self.input_receiver('Enter f(fold), c(call), r(raise).\n >> ')
        if flg in self.__gen_valid_flg(valid_actions):
            if flg == 'f':
                return valid_actions[0]['action'], valid_actions[0]['amount']
            elif flg == 'c':
                return valid_actions[1]['action'], valid_actions[1]['amount']
            elif flg == 'r':
                valid_amounts = valid_actions[2]['amount']
                raise_amount = self.__receive_raise_amount_from_console(valid_amounts['min'], valid_amounts['max'])
                return valid_actions[2]['action'], raise_amount
        else:
            return self.__receive_action_from_console(valid_actions)

    def __gen_valid_flg(self, valid_actions):
        flgs = ['f', 'c']
        is_raise_possible = valid_actions[2]['amount']['min'] != -1
        if is_raise_possible:
            flgs.append('r')
            return flgs

    def __receive_raise_amount_from_console(self, min_amount, max_amount):
        raw_amount = self.input_receiver("valid raise range = [%d, %d]" % (min_amount, max_amount))
        try:
            amount = int(raw_amount)
            if min_amount <= amount and amount <= max_amount:
                return amount
            else:
                print("Invalid raise amount %d. Try again.")
                return self.__receive_raise_amount_from_console(min_amount, max_amount)
        except:
            print("Invalid input received. Try again.")
            return self.__receive_raise_amount_from_console(min_amount, max_amount)

## Board Setup

from pypokerengine.api.game import setup_config, start_poker

init_def_prob = [
    [0.6, 0.2, 0.0, 0.2],
    [0.4, 0.4, 0.1, 0.1],
    [0.1, 0.7, 0.2, 0.0],
    [0.0, 0.6, 0.4, 0.0],
    [0.0, 0.3, 0.7, 0.0]
]

config = setup_config(max_round=10, initial_stack=200, small_blind_amount=1)
config.register_player(name="AI_1", algorithm=HeuristicPlayer(init_def_prob))
config.register_player(name="AI_2", algorithm=HeuristicPlayer(init_def_prob))
config.register_player(name="AI_3", algorithm=HeuristicPlayer(init_def_prob))
config.register_player(name="Human", algorithm=ConsolePlayer())
game_result = start_poker(config, verbose=1)



-- Game start (UUID = satkwiuzdeejybwhtgxgus) --
-- rule --
  - 4 players game
  - 10 round
  - start stack = 200
  -        ante = 0
  - small blind = 1
Enter some key to continue ...
-- Round 1 start (UUID = satkwiuzdeejybwhtgxgus) --
-- hole card --
  - ['S7', 'D5']
-- players information --
  - 0 : AI_1 (kjjlowsuqadtewqadozvvg) => state : participating, stack : 200
  - 1 : AI_2 (gaquqbladtwatezguunsdc) => state : participating, stack : 199
  - 2 : AI_3 (cswsymiotjtohkskgztlbj) => state : participating, stack : 198
  - 3 : Human (satkwiuzdeejybwhtgxgus) => state : participating, stack : 200
Enter some key to continue ...
-- New street start (UUID = satkwiuzdeejybwhtgxgus) --
-- street --
  - preflop
Enter some key to continue ...
Started the round 1
Street "preflop" started. (community card = [])
-- Declare your action (UUID = satkwiuzdeejybwhtgxgus) --
-- valid actions --
  - fold
  - call: 2
  - raise: [3, 200]
-- hole card --
  - ['S7', 'D5']
-- round state --
  - dealer btn : AI

Enter some key to continue ...
"AI_2" declared "raise:5"
[0.0, 0.3, 0.7, 0.0]
-- Game update (UUID = satkwiuzdeejybwhtgxgus) --
-- new action --
  - AI_3 (cswsymiotjtohkskgztlbj) declared call: 5
-- round state --
  - dealer btn : AI_1
  - street : preflop
  - community card : []
  - pot : main = 10, side = []
  - players information
    - 0 : AI_1 (kjjlowsuqadtewqadozvvg) => state : folded, stack : 200
    - 1 : AI_2 (gaquqbladtwatezguunsdc) => state : participating, stack : 195 <= SB
    - 2 : AI_3 (cswsymiotjtohkskgztlbj) => state : participating, stack : 195 <= BB, CURRENT
    - 3 : Human (satkwiuzdeejybwhtgxgus) => state : folded, stack : 200
  - action histories
    - preflop
      - {'action': 'SMALLBLIND', 'amount': 1, 'add_amount': 1, 'player': 'AI_2 (uuid=gaquqbladtwatezguunsdc)'}
      - {'action': 'BIGBLIND', 'amount': 2, 'add_amount': 1, 'player': 'AI_3 (uuid=cswsymiotjtohkskgztlbj)'}
      - {'action': 'FOLD', 'player': 'Human (uuid=satkwiuzdeejybwhtgxgus)'}
      - {'act

Enter some key to continue ...
"AI_2" declared "raise:5"
[0, 0.3, 0.7, 0.0]
-- Game update (UUID = satkwiuzdeejybwhtgxgus) --
-- new action --
  - AI_3 (cswsymiotjtohkskgztlbj) declared raise: 10
-- round state --
  - dealer btn : AI_1
  - street : turn
  - community card : ['HK', 'H2', 'SA', 'S2']
  - pot : main = 31, side = []
  - players information
    - 0 : AI_1 (kjjlowsuqadtewqadozvvg) => state : folded, stack : 200
    - 1 : AI_2 (gaquqbladtwatezguunsdc) => state : participating, stack : 187 <= SB
    - 2 : AI_3 (cswsymiotjtohkskgztlbj) => state : participating, stack : 182 <= BB, CURRENT
    - 3 : Human (satkwiuzdeejybwhtgxgus) => state : folded, stack : 200
  - action histories
    - preflop
      - {'action': 'SMALLBLIND', 'amount': 1, 'add_amount': 1, 'player': 'AI_2 (uuid=gaquqbladtwatezguunsdc)'}
      - {'action': 'BIGBLIND', 'amount': 2, 'add_amount': 1, 'player': 'AI_3 (uuid=cswsymiotjtohkskgztlbj)'}
      - {'action': 'FOLD', 'player': 'Human (uuid=satkwiuzdeejybwhtgxg

Enter some key to continue ...
"AI_2" declared "raise:25"
[0, 0.3, 0.7, 0.0]
-- Game update (UUID = satkwiuzdeejybwhtgxgus) --
-- new action --
  - AI_3 (cswsymiotjtohkskgztlbj) declared call: 25
-- round state --
  - dealer btn : AI_1
  - street : turn
  - community card : ['HK', 'H2', 'SA', 'S2']
  - pot : main = 66, side = []
  - players information
    - 0 : AI_1 (kjjlowsuqadtewqadozvvg) => state : folded, stack : 200
    - 1 : AI_2 (gaquqbladtwatezguunsdc) => state : participating, stack : 167 <= SB
    - 2 : AI_3 (cswsymiotjtohkskgztlbj) => state : participating, stack : 167 <= BB, CURRENT
    - 3 : Human (satkwiuzdeejybwhtgxgus) => state : folded, stack : 200
  - action histories
    - preflop
      - {'action': 'SMALLBLIND', 'amount': 1, 'add_amount': 1, 'player': 'AI_2 (uuid=gaquqbladtwatezguunsdc)'}
      - {'action': 'BIGBLIND', 'amount': 2, 'add_amount': 1, 'player': 'AI_3 (uuid=cswsymiotjtohkskgztlbj)'}
      - {'action': 'FOLD', 'player': 'Human (uuid=satkwiuzdeejybwhtgxg

Enter some key to continue ...
"AI_3" declared "raise:44"
[0, 0.3, 0.7, 0.0]
-- Game update (UUID = satkwiuzdeejybwhtgxgus) --
-- new action --
  - AI_2 (gaquqbladtwatezguunsdc) declared raise: 66
-- round state --
  - dealer btn : AI_1
  - street : river
  - community card : ['HK', 'H2', 'SA', 'S2', 'C6']
  - pot : main = 176, side = []
  - players information
    - 0 : AI_1 (kjjlowsuqadtewqadozvvg) => state : folded, stack : 200
    - 1 : AI_2 (gaquqbladtwatezguunsdc) => state : participating, stack : 101 <= SB, CURRENT
    - 2 : AI_3 (cswsymiotjtohkskgztlbj) => state : participating, stack : 123 <= BB
    - 3 : Human (satkwiuzdeejybwhtgxgus) => state : folded, stack : 200
  - action histories
    - preflop
      - {'action': 'SMALLBLIND', 'amount': 1, 'add_amount': 1, 'player': 'AI_2 (uuid=gaquqbladtwatezguunsdc)'}
      - {'action': 'BIGBLIND', 'amount': 2, 'add_amount': 1, 'player': 'AI_3 (uuid=cswsymiotjtohkskgztlbj)'}
      - {'action': 'FOLD', 'player': 'Human (uuid=satkwiuzdee

...

Enter f(fold), c(call), r(raise).
 >> r
valid raise range = [2, 198]67
-- Game update (UUID = satkwiuzdeejybwhtgxgus) --
-- new action --
  - Human (satkwiuzdeejybwhtgxgus) declared raise: 67
-- round state --
  - dealer btn : AI_2
  - street : flop
  - community card : ['CJ', 'S7', 'HT']
  - pot : main = 73, side = []
  - players information
    - 0 : AI_1 (kjjlowsuqadtewqadozvvg) => state : folded, stack : 200
    - 1 : AI_2 (gaquqbladtwatezguunsdc) => state : participating, stack : 341
    - 2 : AI_3 (cswsymiotjtohkskgztlbj) => state : participating, stack : 55 <= SB
    - 3 : Human (satkwiuzdeejybwhtgxgus) => state : participating, stack : 131 <= BB, CURRENT
  - action histories
    - preflop
      - {'action': 'SMALLBLIND', 'amount': 1, 'add_amount': 1, 'player': 'AI_3 (uuid=cswsymiotjtohkskgztlbj)'}
      - {'action': 'BIGBLIND', 'amount': 2, 'add_amount': 1, 'player': 'Human (uuid=satkwiuzdeejybwhtgxgus)'}
      - {'action': 'FOLD', 'player': 'AI_1 (uuid=kjjlowsuqadtewqadozvvg)'

Enter some key to continue ...
"AI_2" declared "fold:0"
[0.6, 0.2, 0.0, 0.2]
-- Game update (UUID = satkwiuzdeejybwhtgxgus) --
-- new action --
  - AI_3 (cswsymiotjtohkskgztlbj) declared fold: 0
-- round state --
  - dealer btn : AI_3
  - street : preflop
  - community card : []
  - pot : main = 3, side = []
  - players information
    - 0 : AI_1 (kjjlowsuqadtewqadozvvg) => state : participating, stack : 198 <= BB
    - 1 : AI_2 (gaquqbladtwatezguunsdc) => state : folded, stack : 341
    - 2 : AI_3 (cswsymiotjtohkskgztlbj) => state : folded, stack : 55 <= CURRENT
    - 3 : Human (satkwiuzdeejybwhtgxgus) => state : participating, stack : 203 <= SB
  - action histories
    - preflop
      - {'action': 'SMALLBLIND', 'amount': 1, 'add_amount': 1, 'player': 'Human (uuid=satkwiuzdeejybwhtgxgus)'}
      - {'action': 'BIGBLIND', 'amount': 2, 'add_amount': 1, 'player': 'AI_1 (uuid=kjjlowsuqadtewqadozvvg)'}
      - {'action': 'FOLD', 'player': 'AI_2 (uuid=gaquqbladtwatezguunsdc)'}
      - {'acti

Enter some key to continue ...
"Human" declared "call:0"
-- Game update (UUID = satkwiuzdeejybwhtgxgus) --
-- new action --
  - AI_1 (kjjlowsuqadtewqadozvvg) declared call: 0
-- round state --
  - dealer btn : AI_3
  - street : flop
  - community card : ['C9', 'S4', 'ST']
  - pot : main = 200, side = []
  - players information
    - 0 : AI_1 (kjjlowsuqadtewqadozvvg) => state : participating, stack : 100 <= BB, CURRENT
    - 1 : AI_2 (gaquqbladtwatezguunsdc) => state : folded, stack : 341
    - 2 : AI_3 (cswsymiotjtohkskgztlbj) => state : folded, stack : 55
    - 3 : Human (satkwiuzdeejybwhtgxgus) => state : participating, stack : 104 <= SB
  - action histories
    - preflop
      - {'action': 'SMALLBLIND', 'amount': 1, 'add_amount': 1, 'player': 'Human (uuid=satkwiuzdeejybwhtgxgus)'}
      - {'action': 'BIGBLIND', 'amount': 2, 'add_amount': 1, 'player': 'AI_1 (uuid=kjjlowsuqadtewqadozvvg)'}
      - {'action': 'FOLD', 'player': 'AI_2 (uuid=gaquqbladtwatezguunsdc)'}
      - {'action': 'F

Enter f(fold), c(call), r(raise).
 >> c


TypeError: argument of type 'NoneType' is not iterable

In [17]:
np.array([1,2,3])

array([1, 2, 3])

In [32]:
import pypokergui 


In [44]:
import pandas as pd
preflop_eq = pd.read_csv('preflop_equity.csv')
preflop_eq['Cards'] = [str(x) for x in preflop_eq['Cards']]
preflop_eq.set_index('Cards')

def preflop(cards, n):
    """
    Input:
        cards -- in size order, with either suited or unsuited: "AKo" and "KAo"
        num of players -- 3
    Output: Preflop win probability
    """
    print(cards)
    print(preflop_eq['Cards'][69] == 'JJo')
    return float(preflop_eq.loc[(preflop_eq['Cards'] == cards[0]) | (preflop_eq['Cards'] == cards[1])][str(n) + ' plyrs'].tolist()[0].split('%')[0])/100

#preflop(['JJo', 'JJo'], 2)
preflop_eq['Cards'][69]

'JJo '